In [2]:
import os
import pandas as pd

def merge_csv_files():
    # Take user input for usernames
    username1 = input("Enter username 1: ")
    username2 = input("Enter username 2: ")

    # Read the dms_output.csv file
    dms_data = pd.read_csv('../processed_data/dms_output.csv')

    # Filter the data based on the usernames
    filtered_data = dms_data[(dms_data['member_1'] == username1) & (dms_data['member_2'] == username2)]

    if filtered_data.empty:
        print("No matching records found")
        return

    # Get the folder name from the id column of the matched record
    folder_name = filtered_data.iloc[0]['id']

    # Search for the folder with the same name
    folder_path = None
    for root, dirs, files in os.walk('../csv2/'):
        if folder_name in dirs:
            folder_path = os.path.join(root, folder_name)
            break

    if folder_path is None:
        print("Folder not found")
        return
    # Merge all the CSV files in the folder
    merged_data = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path, usecols=["user", "text", "type", "ts"])
            df['ts'] = pd.to_datetime(df['ts'], unit='s')  # Convert ts column to human-readable timestamp
            merged_data = pd.concat([merged_data, df])

    # Save the merged data to a new CSV file
    merged_data.to_csv('merged_conversations.csv', index=False)
    print("CSV files merged successfully!")

# Usage example
merge_csv_files()


CSV files merged successfully!


In [1]:
import os
import pandas as pd
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/messages', methods=['GET'])
def get_messages():
    # Get usernames from the request parameters
    username1 = request.args.get('username1')
    username2 = request.args.get('username2')

    # Read the dms_output.csv file
    dms_data = pd.read_csv('../processed_data/dms_output.csv')

    # Filter the data based on the usernames
    filtered_data = dms_data[(dms_data['member_1'] == username1) & (dms_data['member_2'] == username2)]

    if filtered_data.empty:
        return jsonify({'message': 'No matching records found'})

    # Get the folder name from the id column of the matched record
    folder_name = filtered_data.iloc[0]['id']

    # Search for the folder with the same name
    folder_path = None
    for root, dirs, files in os.walk('../csv2/'):
        if folder_name in dirs:
            folder_path = os.path.join(root, folder_name)
            break

    if folder_path is None:
        return jsonify({'message': 'Folder not found'})

    # Merge all the CSV files in the folder
    merged_data = pd.DataFrame()
    for file in os.listdir(folder_path):
        if file.endswith('.csv'):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path, usecols=["user", "text", "type", "ts"])
            df['ts'] = pd.to_datetime(df['ts'], unit='s')  # Convert ts column to human-readable timestamp
            merged_data = pd.concat([merged_data, df])

    # Filter the merged data based on the usernames
    filtered_messages = merged_data[(merged_data['user'] == username1) | (merged_data['user'] == username2)]

    # Convert the filtered messages to JSON
    messages_json = filtered_messages.to_json(orient='records')

    return messages_json

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [04/Dec/2023 11:47:13] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2023 11:47:13] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2023 11:47:16] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [04/Dec/2023 11:47:16] "GET / HTTP/1.1" 404 -


In [4]:
user_id_to_username = {
    "U05RA88PCPR": "sam",
    "U05S28VB3ED": "admin",
    "U05S28X7ETB": "payagude.m",
    "U05TG4W1YUF": "nikhil993477",
    "U05TQLV3ZC4": "satish",
    "U05TTNUB27P": "sajals1146",
    "U05TZ4E0G5A": "shubhangisharma2411",
    "U05V4DFEHS8": "tanushsethi55"
}

In [5]:
import pandas as pd

# Read the merged_msg.csv file
merged_data = pd.read_csv('merged_msg.csv')

# Take user input for username
username = input("Enter username: ")

# Replace user_id with username in user column
merged_data['user'] = merged_data['user'].replace(user_id_to_username)

# Filter the data based on the username
filtered_data = merged_data[merged_data['user'] == username]

# Print the filtered messages
print(filtered_data)


   user                                               text     type  \
14  sam  Satish joined Slack — take a second to say hello.  message   
21  sam  Satish joined Slack — take a second to say hello.  message   
22  sam  <@U05S28X7ETB> - <https://jobs.ashbyhq.com/ram...  message   
24  sam  Came across the opportunity. Thought you might...  message   
25  sam   Opps! My bad. Just noticed it is for 2024 grades  message   
28  sam  Hello Mrunal - I’ve plans today. Let’s connect...  message   
29  sam     Follow this guy: <https://huryn.substack.com/>  message   
30  sam  <@U05S28X7ETB> - List of PMs : <https://docs.g...  message   
33  sam                                            Means?   message   
35  sam                                     Sure! You can   message   
36  sam  350+ Product Management Questions: <https://ma...  message   
39  sam  _Zubulake v. UBS Warburg L.L.C._ (“_Zubulake V...  message   
40  sam  _Victor Stanley Inc. v. Creative Pipe Inc._U.S...  message   

     

In [ ]:
@app.get("/messages")
def get_messages(username: str):
    # Read the merged_msg.csv file
    merged_data = pd.read_csv('merged_msg.csv')

    # Replace user_id with username in user column
    merged_data['user'] = merged_data['user'].replace(user_id_to_username)

    # Filter the data based on the username
    filtered_data = merged_data[merged_data['user'] == username]

    # Convert the filtered messages to JSON
    messages_json = filtered_data.to_json(orient='records')

    return messages_json

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)
